In [240]:
import torch
import random

In [241]:
# Get random data

In [242]:
def synthetic_data(w, b, num_examples):  #@save
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [243]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2

In [244]:
features, labels = synthetic_data(true_w, true_b, 1000)

In [245]:
print(features[0], labels[0])

tensor([-1.4043, -1.5495]) tensor([6.6670])


In [246]:
def data_iter(batch_size, features, labels):
    num_example = len(features)
    indices = list(range(num_example))
    random.shuffle(indices)
    for i in range(0, num_example, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num_example)])
        yield features[batch_indices], labels[batch_indices]

###  Initial model super parameters

In [247]:
w = torch.normal(0, 0.01, (2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

### Define model

In [248]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b

### Loss Function

In [249]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(-1, 1)) ** 2 / 2

### Optimization 

In [250]:
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()


### Training

In [251]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

In [252]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f"epoch {epoch + 1}, loss{float(train_l.mean()):f}")

epoch 1, loss0.025095
epoch 2, loss0.000087
epoch 3, loss0.000049


### Concise Implement

In [255]:
from torch.utils import data
from torch import nn

In [253]:
true_w = torch.tensor([2, -3.4])
truw_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [254]:
def load_array(features, labels, batch_size, is_train=True):
    dataset = data.TensorDataset(features, labels)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [267]:
data_iter = load_array(features, labels, 10)

In [268]:
net = nn.Sequential(nn.Linear(2, 1))

In [269]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [270]:
loss = nn.MSELoss()

In [271]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [275]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f"epoch {epoch}, loss{l:f}")

epoch 0, loss0.000105
epoch 1, loss0.000104
epoch 2, loss0.000105
